In [50]:
import pandas as pd
import re

FILE = 'salix008_tab.txt'

with open(FILE,'r') as f:
    out = f.read()
    out = re.sub("(  +)", ",", out)
    out = re.sub("\*\*\*\*\*\*\*", "", out)
    with open(f"{FILE}_processed",'w') as o:
        o.write(out)

salix_df = pd.read_csv(f"{FILE}_processed")
wheatfen_df = pd.read_excel("/home/markhaoxiang/Projects/salix/Wheatfen_treecores.xlsx")

In [61]:
wheatfen_df_diameter = wheatfen_df[['ID', 'Diameter/cm']].dropna()
wheatfen_df_diameter['ID'] = wheatfen_df_diameter['ID'].apply(lambda x : x.replace("?",""))
wheatfen_df_diameter

,ID,Diameter/cm
0,WFA01,42.5
1,WFA02,48.5
2,WFP01,34.5
3,WFA03,65.3
4,WFP02,34.0
5,WFA04,58.5
6,WFP03,12.8
7,WFA05,12.0
8,WFP04,15.4
9,WFA07,22.5


In [62]:
salix_df

,year,WFP20B,WFP16A,WFP16B,WFP15A,WFP15B,WFP14A,WFP14B,WFP13A,WFP13B,...,WFP10B,WFP4A,WFP3B,WFP08A,WFP19A,WFP3A,WFP11A,WFP11B,WFP12A2,WFP17A
0,1986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.100,NaN,NaN,NaN,NaN,3.373,NaN
1,1987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.637,NaN,NaN,NaN,NaN,3.995,NaN
2,1988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.244,NaN,NaN,NaN,NaN,3.026,NaN
3,1989,NaN,NaN,NaN,NaN,NaN,1.512,1.951,1.222,NaN,...,NaN,NaN,NaN,1.970,NaN,NaN,NaN,NaN,2.780,NaN
4,1990,NaN,3.408,NaN,NaN,NaN,2.174,1.559,1.342,0.989,...,NaN,NaN,NaN,3.884,NaN,NaN,NaN,NaN,2.803,1.307
5,1991,NaN,2.441,NaN,2.920,3.284,2.183,1.808,4.680,1.868,...,NaN,NaN,NaN,2.161,NaN,NaN,NaN,NaN,2.473,0.799
6,1992,NaN,3.402,3.080,3.135,3.503,2.329,2.598,2.388,2.194,...,NaN,NaN,NaN,3.888,NaN,NaN,NaN,NaN,3.655,1.608
7,1993,NaN,3.139,2.827,2.348,2.739,2.503,2.701,2.190,1.426,...,NaN,NaN,NaN,1.645,NaN,NaN,NaN,NaN,2.475,2.428
8,1994,NaN,4.316,3.863,3.093,2.465,3.061,2.863,2.253,2.685,...,NaN,NaN,NaN,2.286,NaN,NaN,NaN,NaN,2.611,2.623
9,1995,NaN,2.821,2.890,4.164,3.130,2.798,2.896,2.072,0.896,...,NaN,2.801,NaN,1.988,NaN,NaN,NaN,NaN,1.665,3.028
